Import Libraries

In [28]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

Get data from yfinance

In [29]:
def getYFinanceData(ticker, period, interval):
    # get the data from last 60 days until the present
    dataF = yf.download(ticker, period=period, interval=interval)
    df = pd.DataFrame(dataF)

    # use df index, convert DateTime to  a column instead of index
    df.reset_index(inplace=True)

    # delete Adj Close
    df = df.drop(['Adj Close'], axis=1)

    # rename Datetime to "Gmt time"
    df = df.rename(columns={'Datetime':'Gmt time'})
    
    # rename Date to "Gmt time"
    df = df.rename(columns={'Date':'Gmt time'})
    
    # add Datetime "Gmt time" column if it doesn't exist
    if 'Gmt time' not in df.columns:
        df['Gmt time'] = df.index
        
    # set Gmt time as index
    df.set_index('Gmt time', inplace=True)

    df=df[df.High!=df.Low]
    # df.reset_index(inplace=True)
    
    return df

In [30]:
def transformTo4H(df):
    df_4h = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
    for i in range(0, len(df)-3, 4):
        df_4h = pd.concat([df_4h, pd.DataFrame({'Gmt time': [df.index[i]], 'Open': [df['Open'][i]], 'High': [max(df['High'][i:i+4])], 'Low': [min(df['Low'][i:i+4])], 'Close': [df['Close'][i+3]], 'Volume': [sum(df['Volume'][i:i+4])]})], ignore_index=True)
    df_4h.set_index('Gmt time', inplace=True)
    return df_4h

In [31]:
def calculateMACD(df):
    df['MACD'] = ta.macd(df.Close)['MACD_12_26_9']
    df['MACD_HIST'] = ta.macd(df.Close)['MACDh_12_26_9']
    df['MACD_SIGNAL'] = ta.macd(df.Close)['MACDs_12_26_9']
    
    return df

In [32]:
df_1h = getYFinanceData('^NDX', '730d', '1h')
df_1d = getYFinanceData('^NDX', '730d', '1d')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [33]:
df_4h = transformTo4H(df_1h)

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_26889/3611309429.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_26889/3611309429.py:4: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_26889/3611309429.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.il

In [34]:
df_4h

,Open,High,Low,Close,Volume
Gmt time,,,,,
2021-04-29 09:30:00-04:00,14065.977539,14072.416016,13836.099609,13919.929688,258039948
2021-04-29 13:30:00-04:00,13920.004883,13991.261719,13857.300781,13957.274414,204747214
2021-04-30 10:30:00-04:00,13959.013672,13969.001953,13841.281250,13873.360352,248856502
2021-04-30 14:30:00-04:00,13873.078125,13954.286133,13792.721680,13796.040039,242491402
2021-05-03 11:30:00-04:00,13794.383789,13833.763672,13786.997070,13810.313477,210995911
...,...,...,...,...,...
2024-03-19 14:30:00-04:00,18022.703125,18087.119141,17974.912109,18044.521484,253463979
2024-03-20 11:30:00-04:00,18042.394531,18238.904297,18003.416016,18236.775391,369048064
2024-03-20 15:30:00-04:00,18237.611328,18464.183594,18195.208984,18462.585938,345322169


In [35]:
df_1d

,Open,High,Low,Close,Volume
Gmt time,,,,,
2021-04-29,14063.940430,14073.480469,13836.099609,13970.200195,4928030000
2021-04-30,13857.790039,13969.209961,13838.820312,13860.759766,4792460000
2021-05-03,13923.040039,13955.000000,13784.660156,13799.719727,4761430000
2021-05-04,13680.629883,13705.379883,13396.110352,13544.669922,5879730000
2021-05-05,13649.120117,13676.860352,13478.580078,13503.370117,4535690000
...,...,...,...,...,...
2024-03-18,18024.240234,18124.320312,17973.089844,17985.009766,5025040000
2024-03-19,17902.519531,18039.820312,17809.150391,18032.199219,4758740000
2024-03-20,18067.699219,18249.119141,18003.240234,18240.109375,5298380000


In [36]:
df_4h = calculateMACD(df_4h)
df_1d = calculateMACD(df_1d)

In [37]:
df_4h

,Open,High,Low,Close,Volume,MACD,MACD_HIST,MACD_SIGNAL
Gmt time,,,,,,,,
2021-04-29 09:30:00-04:00,14065.977539,14072.416016,13836.099609,13919.929688,258039948,NaN,NaN,NaN
2021-04-29 13:30:00-04:00,13920.004883,13991.261719,13857.300781,13957.274414,204747214,NaN,NaN,NaN
2021-04-30 10:30:00-04:00,13959.013672,13969.001953,13841.281250,13873.360352,248856502,NaN,NaN,NaN
2021-04-30 14:30:00-04:00,13873.078125,13954.286133,13792.721680,13796.040039,242491402,NaN,NaN,NaN
2021-05-03 11:30:00-04:00,13794.383789,13833.763672,13786.997070,13810.313477,210995911,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-03-19 14:30:00-04:00,18022.703125,18087.119141,17974.912109,18044.521484,253463979,4.473737,-17.873293,22.347030
2024-03-20 11:30:00-04:00,18042.394531,18238.904297,18003.416016,18236.775391,369048064,24.127792,1.424610,22.703182
2024-03-20 15:30:00-04:00,18237.611328,18464.183594,18195.208984,18462.585938,345322169,57.264685,27.649202,29.615483


In [38]:
df_1d

# add Date column
df_1d['Date'] = df_1d.index


In [39]:
# convert the index to a pandas DatetimeIndex
df_4h.index = pd.to_datetime(df_4h.index)

# create a new column called "Date", strip the time and store the date here
df_4h['Date'] = df_4h.index.date

df_4h

,Open,High,Low,Close,Volume,MACD,MACD_HIST,MACD_SIGNAL,Date
Gmt time,,,,,,,,,
2021-04-29 09:30:00-04:00,14065.977539,14072.416016,13836.099609,13919.929688,258039948,NaN,NaN,NaN,2021-04-29
2021-04-29 13:30:00-04:00,13920.004883,13991.261719,13857.300781,13957.274414,204747214,NaN,NaN,NaN,2021-04-29
2021-04-30 10:30:00-04:00,13959.013672,13969.001953,13841.281250,13873.360352,248856502,NaN,NaN,NaN,2021-04-30
2021-04-30 14:30:00-04:00,13873.078125,13954.286133,13792.721680,13796.040039,242491402,NaN,NaN,NaN,2021-04-30
2021-05-03 11:30:00-04:00,13794.383789,13833.763672,13786.997070,13810.313477,210995911,NaN,NaN,NaN,2021-05-03
...,...,...,...,...,...,...,...,...,...
2024-03-19 14:30:00-04:00,18022.703125,18087.119141,17974.912109,18044.521484,253463979,4.473737,-17.873293,22.347030,2024-03-19
2024-03-20 11:30:00-04:00,18042.394531,18238.904297,18003.416016,18236.775391,369048064,24.127792,1.424610,22.703182,2024-03-20
2024-03-20 15:30:00-04:00,18237.611328,18464.183594,18195.208984,18462.585938,345322169,57.264685,27.649202,29.615483,2024-03-20


In [40]:
# Convert 'Date' to datetime
df_4h['Date'] = pd.to_datetime(df_4h['Date'])
df_1d['Date'] = pd.to_datetime(df_1d['Date'])

# Set 'Date' as index for df_1d
df_1d.set_index('Date', inplace=True)

# Join df_4h and df_1d on index
df_4h = df_4h.join(df_1d[['MACD', 'MACD_HIST', 'MACD_SIGNAL']].add_suffix('_1D'), on='Date')

df_4h

,Open,High,Low,Close,Volume,MACD,MACD_HIST,MACD_SIGNAL,Date,MACD_1D,MACD_HIST_1D,MACD_SIGNAL_1D
Gmt time,,,,,,,,,,,,
2021-04-29 09:30:00-04:00,14065.977539,14072.416016,13836.099609,13919.929688,258039948,NaN,NaN,NaN,2021-04-29,NaN,NaN,NaN
2021-04-29 13:30:00-04:00,13920.004883,13991.261719,13857.300781,13957.274414,204747214,NaN,NaN,NaN,2021-04-29,NaN,NaN,NaN
2021-04-30 10:30:00-04:00,13959.013672,13969.001953,13841.281250,13873.360352,248856502,NaN,NaN,NaN,2021-04-30,NaN,NaN,NaN
2021-04-30 14:30:00-04:00,13873.078125,13954.286133,13792.721680,13796.040039,242491402,NaN,NaN,NaN,2021-04-30,NaN,NaN,NaN
2021-05-03 11:30:00-04:00,13794.383789,13833.763672,13786.997070,13810.313477,210995911,NaN,NaN,NaN,2021-05-03,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-19 14:30:00-04:00,18022.703125,18087.119141,17974.912109,18044.521484,253463979,4.473737,-17.873293,22.347030,2024-03-19,109.783386,-42.618900,152.402285
2024-03-20 11:30:00-04:00,18042.394531,18238.904297,18003.416016,18236.775391,369048064,24.127792,1.424610,22.703182,2024-03-20,120.417353,-25.587946,146.005299
2024-03-20 15:30:00-04:00,18237.611328,18464.183594,18195.208984,18462.585938,345322169,57.264685,27.649202,29.615483,2024-03-20,120.417353,-25.587946,146.005299


In [41]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df_4h.copy()

# Create a subplot with 2 rows and 1 column
fig = make_subplots(rows=2, cols=1)

# Add candlestick plot to the first row of the subplot
fig.add_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']), row=1, col=1)

# Add MACD plots to the second row of the subplot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['MACD'], mode='lines', name='MACD', line=dict(color='blue')), row=2, col=1)


# Add MACD plots to the second row of the subplot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['MACD_1D'], mode='lines', name='MACD_1D', line=dict(color='orange')), row=2, col=1)


fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(autosize=False, width=1000, height=600,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")

fig.show()

In [42]:
df = df_4h

In [43]:
signal = [0]*len(df)

for row in range(1, len(df)):
  previous_macd_4h = df.iloc[row-1]['MACD']
  current_macd_4h = df.iloc[row]['MACD']
  
  current_macd_1d = df.iloc[row]['MACD_1D']

  if (previous_macd_4h < 0 and current_macd_4h > 0 and current_macd_1d > 0):
    # print(f"Buy at {df.iloc[row,0]}")
    hasSetup = True
    signal[row]=1
  elif (previous_macd_4h > 0 and current_macd_4h < 0 and current_macd_1d < 0):
    # print(f"Sell at {df.iloc[row,0]}")
    hasSetup = True
    signal[row]=-1
  else:
    hasSetup = False
    signal[row]=0

df['signal_4h'] = signal

In [44]:
def TotalSignal(l):
    if (
        df['signal_4h'][l]==1
    ):
            return 2
    if (
        df['signal_4h'][l]==-1
    ):
            return 1
    return 0
        
TotSignal = [0]*len(df)
for row in range(0, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_26889/2829009616.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_26889/2829009616.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [45]:
df[df.TotalSignal!=0].count()

Open              11
High              11
Low               11
Close             11
Volume            11
MACD              11
MACD_HIST         11
MACD_SIGNAL       11
Date              11
MACD_1D           11
MACD_HIST_1D      11
MACD_SIGNAL_1D    11
signal_4h         11
TotalSignal       11
dtype: int64

In [46]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [47]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=0
dfpl = df[st:st+ len(df)]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                # go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                #            line=dict(color='blue', width=1), 
                #            name="VWAP"), 
                # go.Scatter(x=dfpl.index, y=dfpl['BBL.0'], 
                #            line=dict(color='green', width=1), 
                #            name="BBL"),
                # go.Scatter(x=dfpl.index, y=dfpl['BBU.0'], 
                #            line=dict(color='green', width=1), 
                #            name="BBU")
                # go.Scatter(x=dfpl.index, y=dfpl['50_MA'],
                #             line=dict(color='red', width=1), 
                #             name="50_MA"),
      ])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")

# set figure size
fig.update_layout(
    autosize=False,
    width=1600,
    height=1000,
    margin=dict(l=50,r=50,b=100,t=100,pad=4)
)

fig.show()

In [48]:
df['index_entry'] = df.index
df = df.iloc[:-1]
df=df[df.Volume!=0]

In [49]:
dfpl = df[:].copy()
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=7)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

In [50]:
dfpl

,Open,High,Low,Close,Volume,MACD,MACD_HIST,MACD_SIGNAL,Date,MACD_1D,MACD_HIST_1D,MACD_SIGNAL_1D,signal_4h,TotalSignal,pointposbreak,index_entry,ATR
Gmt time,,,,,,,,,,,,,,,,,
2021-04-29 09:30:00-04:00,14065.977539,14072.416016,13836.099609,13919.929688,258039948,NaN,NaN,NaN,2021-04-29,NaN,NaN,NaN,0,0,NaN,2021-04-29 09:30:00-04:00,NaN
2021-04-29 13:30:00-04:00,13920.004883,13991.261719,13857.300781,13957.274414,204747214,NaN,NaN,NaN,2021-04-29,NaN,NaN,NaN,0,0,NaN,2021-04-29 13:30:00-04:00,NaN
2021-04-30 10:30:00-04:00,13959.013672,13969.001953,13841.281250,13873.360352,248856502,NaN,NaN,NaN,2021-04-30,NaN,NaN,NaN,0,0,NaN,2021-04-30 10:30:00-04:00,NaN
2021-04-30 14:30:00-04:00,13873.078125,13954.286133,13792.721680,13796.040039,242491402,NaN,NaN,NaN,2021-04-30,NaN,NaN,NaN,0,0,NaN,2021-04-30 14:30:00-04:00,NaN
2021-05-03 11:30:00-04:00,13794.383789,13833.763672,13786.997070,13810.313477,210995911,NaN,NaN,NaN,2021-05-03,NaN,NaN,NaN,0,0,NaN,2021-05-03 11:30:00-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-19 10:30:00-04:00,17834.369141,18034.796875,17830.044922,18022.447266,312509876,-1.102113,-27.917466,26.815353,2024-03-19,109.783386,-42.618900,152.402285,0,0,NaN,2024-03-19 10:30:00-04:00,199.971380
2024-03-19 14:30:00-04:00,18022.703125,18087.119141,17974.912109,18044.521484,253463979,4.473737,-17.873293,22.347030,2024-03-19,109.783386,-42.618900,152.402285,1,2,17974.912009,2024-03-19 14:30:00-04:00,187.433616
2024-03-20 11:30:00-04:00,18042.394531,18238.904297,18003.416016,18236.775391,369048064,24.127792,1.424610,22.703182,2024-03-20,120.417353,-25.587946,146.005299,0,0,NaN,2024-03-20 11:30:00-04:00,194.298568


In [51]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.03
    mysize = initsize
    risk = 0
    latestEntry = 0
    
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2.5
        perc = 0.02
        lastEntryIndex = 0
        
        if len(self.trades)>0:
            
            # close the position if the amount lost exceeds 3% of the account balance
            if (self.position.pl < -1 * self.equity * 0.015):
                self.position.close()
                print("Close position due to loss")
                print("------")
            
            # # if the pl is greater than the stop loss amount, then move the stop loss to entry
            # if (self.position.pl > self.risk):
            #     # adjust the stop loss to entry
            #     self.position.sl = self.latestEntry
        
            if self.data['index_entry'][-1]:
                self.trades[-1].close()
                self.lastEntryIndex = self.data['index_entry'][-1]
            if self.trades[-1].is_long:
                self.trades[-1].close()
                self.lastEntryIndex = self.data['index_entry'][-1]
            elif self.trades[-1].is_short:
                self.trades[-1].close()
                self.lastEntryIndex = self.data['index_entry'][-1]
        
        if self.signal!=0 and len(self.trades)==0:  
            sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1-perc)
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            entry = self.data.Close[-1]
            self.risk = entry - sl1
            self.latestEntry = entry
            self.lastEntryIndex = self.data['index_entry'][-1]
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal!=0 and len(self.trades)==0:         
            sl1 = sl1 = max(self.data.High[-1],self.data.High[-2])*(1+perc)
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            entry = self.data.Close[-1]
            self.risk = sl1 - entry
            self.latestEntry = entry
            self.lastEntryIndex = self.data['index_entry'][-1]
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=50000, margin=1/100, commission=.000)
stat = bt.run()
stat

Close position due to loss
------
Close position due to loss
------


Start                     2021-04-29 09:30...
End                       2024-03-21 12:30...
Duration                   1057 days 03:00:00
Exposure Time [%]                    1.653543
Equity Final [$]                 48241.730078
Equity Peak [$]                  50265.040625
Return [%]                           -3.51654
Buy & Hold Return [%]               31.592111
Return (Ann.) [%]                   -1.234907
Volatility (Ann.) [%]                8.538627
Sharpe Ratio                        -0.144626
Sortino Ratio                       -0.186297
Calmar Ratio                        -0.141966
Max. Drawdown [%]                   -8.698583
Avg. Drawdown [%]                   -8.162718
Max. Drawdown Duration      378 days 20:00:00
Avg. Drawdown Duration      354 days 20:00:00
# Trades                                   11
Win Rate [%]                        54.545455
Best Trade [%]                       2.247761
Worst Trade [%]                     -2.989956
Avg. Trade [%]                    

In [52]:
bt = Backtest(df, MyStrat, cash=50000, margin=1/100, commission=0.00)
stat = bt.run()

Close position due to loss
------
Close position due to loss
------


In [53]:
print(stat)

Start                     2021-04-29 09:30...
End                       2024-03-21 12:30...
Duration                   1057 days 03:00:00
Exposure Time [%]                    1.653543
Equity Final [$]                 48241.730078
Equity Peak [$]                  50265.040625
Return [%]                           -3.51654
Buy & Hold Return [%]               31.592111
Return (Ann.) [%]                   -1.234907
Volatility (Ann.) [%]                8.538627
Sharpe Ratio                        -0.144626
Sortino Ratio                       -0.186297
Calmar Ratio                        -0.141966
Max. Drawdown [%]                   -8.698583
Avg. Drawdown [%]                   -8.162718
Max. Drawdown Duration      378 days 20:00:00
Avg. Drawdown Duration      354 days 20:00:00
# Trades                                   11
Win Rate [%]                        54.545455
Best Trade [%]                       2.247761
Worst Trade [%]                     -2.989956
Avg. Trade [%]                    

In [54]:
bt.plot()


ValueError: Invalid value for `superimpose`: Upsampling not supported.